In [4]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from collections import Counter
import json
import warnings
import re
warnings.filterwarnings('ignore')
import random
import statistics

from scipy.linalg import solve
import scipy
# import igraph as ig
from scipy.sparse import linalg as sla

In [5]:
def get_mat_file_info(filename):
    mat_file=loadmat(filename)
    # Get the number of files
    files=mat_file['dbs']['filename'].shape[1]
    file_names=[]
    for fi in range(files):
        file_names.append(mat_file['dbs']['filename'][0][fi][0])
    return file_names


# Choose file
## focusing on Head-tail for now 
filename='./data/dbs_Head-Tail_20191013.mat'


# # Get filenames
files=get_mat_file_info(filename)

#dividing up the groups into 4
CCvsCC_female = []
FCvsCC_female = [] 
CCvsCC_male = [] 
FCvsCC_male = []
NacvsCC_male = []

for file in files:
#     print(file)
    if 'Female-CCvsCC' in file:
        CCvsCC_female.append(file)
    elif 'Female-FCvsCC' in file:
        FCvsCC_female.append(file)
    elif 'Male-CCvsCC' in file:
        CCvsCC_male.append(file) 
    elif 'Male-FCvsCC' in file:
        FCvsCC_male.append(file)  
    else:
        NacvsCC_male.append(file)
all_treatments = [CCvsCC_female, FCvsCC_female, CCvsCC_male, FCvsCC_male, NacvsCC_male]
print("Number of files for female-CCvsCC: " + str(len(CCvsCC_female)))
print("Number of files for female-FCvsCC: " + str(len(FCvsCC_female)))
print("Number of files for male-CCvsCC: " + str(len(CCvsCC_male)))
print("Number of files for male-FCvsCC: " + str(len(FCvsCC_male)))
print("Number of files for male-NACvsCC: " + str(len(NacvsCC_male)))


Number of files for female-CCvsCC: 10
Number of files for female-FCvsCC: 12
Number of files for male-CCvsCC: 12
Number of files for male-FCvsCC: 8
Number of files for male-NACvsCC: 8


In [6]:
#from https://rdrr.io/cran/igraph/src/R/structural.properties.R#sym-bonpow.dense
def bonacich_power_centrality(graph,exponent,tol,rescale):

    vg = 6 #number of mice

    #convert to matrix
    d = nx.to_scipy_sparse_matrix(graph)

    #adjacency matrix of size 6 (number of mice)
    id =np.identity(vg)
    
    degree_out = graph.out_degree()
    xarr = np.array(degree_out)
    degree_out_array = np.array([xarr[0][1],xarr[1][1],xarr[2][1],xarr[3][1],xarr[4][1],xarr[5][1]])
    degree_out_array = degree_out_array.astype(np.float)
    

    if round(np.linalg.det(id-(exponent*d))) == 0:
        return {}
    
    lu = sla.splu(id-(exponent*d))
    x = lu.solve(degree_out_array)
    
    if rescale:
        x = x/sum(x)
    else:
        x = x * (np.sqrt(vg/sum(x**2)))
        
    d =  {}
    for i in range(6):
        d[xarr[i][0]] = x[i]
    return d

In [7]:
def matlab_rand_net(in_deg,out_deg):

    n = len(in_deg)
    k = np.sum(in_deg).astype(np.int64)
    inInv = np.zeros((k,))
    outInv = np.zeros((k,))
    iIn = 0
    iOut = 0;

    for i in range(n):
        inInv[iIn:iIn+in_deg[i] ] = i+1
        outInv[iOut:iOut+out_deg[i] ] = i+1
        iIn = iIn+in_deg[i]
        iOut = iOut+out_deg[i]
    cij = np.eye(n).astype(np.int64)
    flag=-5
    edges = np.vstack([np.transpose(outInv[:k]), np.transpose(inInv[np.arange(k)])]).astype(np.int64)
    edges=edges-1
    for i in range(k):
        if cij[edges[0,i],edges[1,i]]==1:
            warningCounter = 1
            while 1:
                switchTo = np.random.randint(k)
                s=0
                if cij[edges[0,i],edges[1,switchTo]] ==0 and cij[edges[0,switchTo],edges[1,i]] ==0:
                    cij[edges[0,i],edges[1,switchTo]] = 1
                    if switchTo <= i:
                        cij[edges[0,switchTo],edges[1,switchTo]] = 0
                        cij[edges[0,switchTo],edges[1,i]] = 1
                    temp = edges[1,i]
                    edges[1,i] = edges[1,switchTo]
                    edges[1,switchTo] = temp;
                    break
                
                warningCounter = warningCounter+1;
                if warningCounter == 2*k^2:
                    flag = False
                    return cij,flag
        else:
            cij[edges[0,i],edges[1,i]] = 1
    cij = cij - np.eye(n)
    if flag==-5:
        flag=True
    return cij,flag

def nx_from_connectivity(A):
    G=nx.DiGraph()
    for i in range(np.shape(A)[0]):
        G.add_node(i)
    edges=np.where(A)
    for i in range(len(edges[0])):
        G.add_edge(edges[0][i],edges[1][i])
    return G

## Functions that calculate Z-Scores (Total or Divided in 2 groups)

In [8]:
def calculate_score(raw, network_measure_func, H,nrand, exponent, tol,rescale):
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
    
    #calculate network measure of 10000 random networks
    
    rand_values=np.zeros((nrand,))
    if network_measure_func == "NA":
        b = (bonacich_power_centrality(H,exponent,tol,rescale)).values()
    else:
        b = (network_measure_func(H)).values()
    
    raw_value=np.nanmean(list(b))
    
    if raw:
        return raw_value
    
    if network_measure_func == "NA" and not raw:
#         print("z-score all bonacich")
        results=np.zeros((6,1000))
        for i in range(nrand):
            done=False
            while not done:
                try:
                    #results[:,i] gets column i
                    results[:,i]=np.array(list(bonacich_power_centrality(nx.directed_configuration_model(din, dout, create_using=nx.DiGraph()), exponent, tol, rescale).values()))
                    done=True
                except:
                    False

        z_score = (np.mean((results[:,1]-np.mean(results[:,1:],1))/np.std(results[:,1:])))
        return z_score
    
    
    for i in range(nrand):
        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        D = nx_from_connectivity(rnet)
        if network_measure_func == "NA":
            b = (bonacich_power_centrality(D,exponent,tol,rescale)).values()
        else:
            b = (network_measure_func(D)).items()
                
        rand_values[i] = np.nanmean(list(b))
    
    if np.round(statistics.stdev(rand_values)*1000) != 0:
        z_score = (raw_value- np.nanmean(rand_values)) / (np.std(rand_values)) 
    else:
        z_score = np.nan
    return z_score
    
def measures_CCvsCC(raw, network_measure_func, H, nrand, exponent, tol,rescale):

    #calculate actual network measures
    d_network = {'CC-1': [], 'CC-2': []}
    if network_measure_func == "NA":
        H_items = (bonacich_power_centrality(H,exponent,tol,rescale)).items()
    else:
        H_items = (network_measure_func(H)).items()
        
    for key, value in H_items:   
        if key in ['CC1_1', 'CC1_2', 'CC1_3']:
            d_network['CC-1'].append(value)
        else:
            d_network['CC-2'].append(value)
    
    raw_value_CC1=np.nanmean(list(d_network['CC-1']))
    raw_value_CC2=np.nanmean(list(d_network['CC-2'])) 
  
    if raw:
        return [raw_value_CC1, raw_value_CC2]
    
    #calculate random network measures 
    rand_values_CC1=np.zeros((nrand,))
    rand_values_CC2=np.zeros((nrand,))

    
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
  
    for i in range(nrand):
        #reset random dicts every iteration 
        d_random = {'CC-1': [], 'CC-2': []}
    
        
        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        
        random_nodes = list(random.sample(H.nodes,len(H.nodes)))
        map_nodes = {} 
        for count, node in enumerate(random_nodes):
            map_nodes[count] = node
        
        D = nx_from_connectivity(rnet)
        D =nx.relabel_nodes(D,map_nodes)
        
        if network_measure_func == "NA":
            d_items = (bonacich_power_centrality(D,exponent,tol,rescale)).items()
        else:
            d_items = (network_measure_func(D)).items()
        
        #calculate network measure
        for key, value in d_items:
            if key in ['CC1_1', 'CC1_2', 'CC1_3']:
                d_random['CC-1'].append(value)
            else:
                d_random['CC-2'].append(value)
            
        rand_values_CC1[i] = np.nanmean(list(d_random['CC-1']))
        rand_values_CC2[i] = np.nanmean(list(d_random['CC-1']))
        
    
    if np.round(statistics.stdev(rand_values_CC1)*1000) != 0:
        z_score_CC1 = (raw_value_CC1 -np.nanmean(rand_values_CC1)) / (np.std(rand_values_CC1)) 
    else:
        z_score_CC1 = np.nan

    
    if np.round(statistics.stdev(rand_values_CC2)*1000) != 0:
        z_score_CC2 = (raw_value_CC2- np.nanmean(rand_values_CC2)) / (np.std(rand_values_CC2)) 
    else:
        z_score_CC2 = np.nan
        
    return [z_score_CC1, z_score_CC2]
        
        
def measures_NACvsCC(raw, network_measure_func, H, nrand, exponent, tol,rescale):
    
    #calculate actual network measures
    d_network = {'CC': [], 'NAC': []}
    
    if network_measure_func == "NA":
        H_items = (bonacich_power_centrality(H,exponent,tol,rescale)).items()
    else:
        H_items = (network_measure_func(H)).items()
#     print(H_items)
    for key, value in H_items:   
#         print(key)
        if "CC" in str(key):
            d_network['CC'].append(value)
        elif "Nac" in str(key):
            d_network['NAC'].append(value)
    
    raw_value_NAC=np.nanmean(list(d_network['NAC']))
    raw_value_CC=np.nanmean(list(d_network['CC'])) 

    if raw:
        return [raw_value_CC, raw_value_NAC]
    
    #calculate random network measures 
    rand_values_CC=np.zeros((nrand,))
    rand_values_NAC=np.zeros((nrand,))
    
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
  
    for i in range(nrand):
        #reset random dicts every iteration 
        d_random = {'CC': [], 'NAC': []}
        

        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        
        random_nodes = list(random.sample(H.nodes,len(H.nodes)))
        map_nodes = {} 
        for count, node in enumerate(random_nodes):
            map_nodes[count] = node
        
        D = nx_from_connectivity(rnet)
        D =nx.relabel_nodes(D,map_nodes)
        
        
        
        if network_measure_func == "NA":
            d_items = (bonacich_power_centrality(D,exponent,tol,rescale)).items()
        else:
            d_items = (network_measure_func(D)).items()  
       
        #calculate network measure
        for key, value in d_items:
            if "CC" in key:
                d_random['CC'].append(value)
            elif "Nac" in key:
                d_random['NAC'].append(value)
            
        rand_values_CC[i] = np.nanmean(list(d_random['CC']))
        rand_values_NAC[i] = np.nanmean(list(d_random['NAC']))

    if np.round(statistics.stdev(rand_values_CC)*1000) != 0:
        z_score_CC = (raw_value_CC -np.nanmean(rand_values_CC)) / (np.std(rand_values_CC)) 
    else:
        z_score_CC = np.nan

    if np.round(statistics.stdev(rand_values_NAC)*1000) != 0:
        z_score_NAC = (raw_value_NAC- np.nanmean(rand_values_NAC)) / (np.std(rand_values_NAC)) 
    else:
        z_score_NAC = np.nan
        
    return [z_score_CC, z_score_NAC]


def measures_FCvsCC(raw, network_measure_func, H, nrand, exponent, tol,rescale):
    
    #calculate actual network measures
    d_network = {'CC': [], 'FC': []}
    if network_measure_func == "NA":
        H_items = (bonacich_power_centrality(H,exponent,tol,rescale)).items()
    else:
        H_items = (network_measure_func(H)).items()
    
    for key, value in H_items:   
        if "FC" in key:
            d_network['FC'].append(value)
        elif "CC" in key:
            d_network['CC'].append(value)
    
    raw_value_FC=np.nanmean(list(d_network['FC']))
    raw_value_CC=np.nanmean(list(d_network['CC'])) 

    if raw: 
        return [raw_value_CC, raw_value_FC]
    #calculate random network measures 
    rand_values_CC=np.zeros((nrand,))
    rand_values_FC=np.zeros((nrand,))
    
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
  
    for i in range(nrand):
        #reset random dicts every iteration 
        d_random = {'CC': [], 'FC': []}
        
        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        
        random_nodes = list(random.sample(H.nodes,len(H.nodes)))
        map_nodes = {} 
        for count, node in enumerate(random_nodes):
            map_nodes[count] = node
        
        D = nx_from_connectivity(rnet)
        D =nx.relabel_nodes(D,map_nodes)
        
        if network_measure_func == "NA":
            d_items = (bonacich_power_centrality(D,exponent,tol,rescale)).items()
        else:
            d_items = (network_measure_func(D)).items()
        
        #calculate network measure
        for key, value in d_items:
            if "CC" in key:
                d_random['CC'].append(value)
            elif "FC" in key:
                d_random['FC'].append(value)
            
        rand_values_CC[i] = np.nanmean(list(d_random['CC']))
        rand_values_FC[i] = np.nanmean(list(d_random['FC']))
        
    
    if np.round(statistics.stdev(rand_values_CC)*1000) != 0:
        z_score_CC = (raw_value_CC -np.nanmean(rand_values_CC)) / (np.std(rand_values_CC)) 
    else:
        z_score_CC = np.nan

    
    if np.round(statistics.stdev(rand_values_FC)*1000) != 0:
        z_score_FC = (raw_value_FC- np.nanmean(rand_values_FC)) / (np.std(rand_values_FC)) 
    else:
        z_score_FC = np.nan
        
    return [z_score_CC, z_score_FC]



In [9]:
def load_iterations_from_mat_file(filename,file):
    mat_file=loadmat(filename)
    # Get the number of files
    files=mat_file['dbs']['filename'].shape[1]
    file_names=[]
    for fi in range(files):
        file_names.append(mat_file['dbs']['filename'][0][fi][0])
    file_num=np.where(np.array(file_names)==file)[0][0]
        
    if len(mat_file['dbs']['networks'][0][file_num].shape)<3:
        mat_file['dbs']['networks'][0][file_num]=mat_file['dbs']['networks'][0][file_num][:,:,np.newaxis]
        
    G_list=[]
    # Get the number of mice
    nmice=mat_file['dbs']['networks'][0][file_num][:,:,0].shape[0]
    # Get the ids
    ids_t=mat_file['dbs']['ids'][0][file_num][0]
    ids=[]
    for id in range(nmice):
        ids.append(ids_t[id][0])
    types=np.unique(ids)
    type_count=np.ones((len(types),),np.int64)
    mapping = {}
    for i in range(nmice):
        mapping[i]=ids[i]+'_'+str(type_count[np.where(types==ids[i])[0]][0])
        type_count[np.where(types==ids[i])[0]]+=1
    # Get the number of iterations
    iters=mat_file['dbs']['networks'][0][file_num].shape[2]
    
    
    # Make the graph
    for iter in range(iters):
        mat_net=mat_file['dbs']['networks'][0][file_num][:,:,iter]
        mat_net[mat_net>0]=1
        G=nx.from_numpy_matrix(mat_net,create_using=nx.DiGraph())
        G=nx.relabel_nodes(G,mapping)
        G_list.append(G)
    return G_list



def get_values(treatment_file, interaction_file):
    lst = [] 
    if 'Female-CCvsCC' in treatment_file:
        lst = ['Female', 'CCvsCC']
    elif 'Female-FCvsCC' in treatment_file:
        lst = ['Female', 'FCvsCC']
    elif 'Male-CCvsCC' in treatment_file:
        lst = ['Male', 'CCvsCC']
    elif 'Male-FCvsCC' in treatment_file:
        lst = ['Male', 'FCvsCC']
    elif 'NacvsCC' in treatment_file:
        lst = ['Male', 'NACvsCC']
    if 'Head-Tail' in interaction_file:
        return lst + ['Head-Tail']
    if 'Head-Head' in interaction_file:
        return lst + ['Head-Head']
    if 'Centers-Close' in interaction_file:
        return lst + ['Centers-Close']
    if 'Oriented-Towards' in interaction_file:
        return lst + ['Oriented-Towards']

   
    
def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, random_iterations):
    date = "20191310"
    print("Properties of current analysis:")
    print("Raw: ", str(raw))
    print("Split groups calculation: ", str(not All))
    print("Network Measure: ", str(network_measures[0][0]))
    print("----------------------")
    df_all_zscore = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File'])
    df_split_zscore = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Care', 'Group', 'File'])
 
    for treatment in treatments: 
    
        # Get filenames
        files=get_mat_file_info(treatment)
    
        #dividing up the groups into 4
        CCvsCC_female, FCvsCC_female, CCvsCC_male, FCvsCC_male, NACvsCC_male  = ([] for i in range(5))
        for file in files:
            if 'Female-CCvsCC' in file:
                CCvsCC_female.append(file)
            elif 'Female-FCvsCC' in file:
                FCvsCC_female.append(file)
            elif 'Male-CCvsCC' in file:
                CCvsCC_male.append(file) 
            elif 'Male-FCvsCC' in file:
                FCvsCC_male.append(file)
            elif 'Male-NacvsCC' in file:
                NACvsCC_male.append(file)
        all_treatments = [NACvsCC_male, CCvsCC_female, FCvsCC_female, CCvsCC_male, FCvsCC_male]
        total_iterations = 0
        t = 1
        for x in range(len(all_treatments)):
            mean_trial_raw = {}
            mean_trial_zscore = {} 
            for i in range(len(all_treatments[x])):
                file=all_treatments[x][i]
                print(file)
                print(treatment)
                

                # Get the network measures
                G_list=load_iterations_from_mat_file(treatment,file)
                
                
                df_all_temp = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File'])
                df_split_temp_1 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Care', 'Group', 'File'])
                df_split_temp_2 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Care', 'Group', 'File'])

                
                for ii in range(len(G_list)):
                    gender, strain, interaction = get_values(all_treatments[x][i], treatment)

                    total_iterations += 1

                    G=G_list[ii]
                    if len(G_list) == 26 and ii == 7:
                        continue
                     
                    
                    if All:
                        #calculate z-score for all network, and add to df_zscore
                        for i in range(len(network_measures)):
                            z_score = calculate_score(raw, network_measures[i][1],G,random_iterations,exponent,tol,rescale) 
                            df_all_temp.loc[len(df_all_temp)]=[network_measures[i][0], z_score, interaction, gender, strain, t, file]

                    else:
#                         print(len(G_list))
#                         print(strain, gender, interaction)
                            
                        #split z-score into FC and CC or CC1 and CC2, add to df_zscore  
                        if strain == "NACvsCC":
                            score_CC, score_NAC = measures_NACvsCC(raw, network_measures[0][1], G, random_iterations, exponent,tol,rescale)
                            df_split_temp_1.loc[len(df_split_temp_1)]=[network_measures[0][0], score_CC, interaction, gender, strain, "CC", t, file]
                            df_split_temp_2.loc[len(df_split_temp_2)]=[network_measures[0][0], score_NAC, interaction, gender, strain, "NAC", t, file]
                        elif strain == "CCvsCC":
                            score_CC1, score_CC2 = measures_CCvsCC(raw, network_measures[0][1], G, random_iterations, exponent,tol,rescale)
                            df_split_temp_1.loc[len(df_split_temp_1)]=[network_measures[0][0], score_CC1, interaction, gender, strain, "CC1", t, file]
                            df_split_temp_2.loc[len(df_split_temp_2)]=[network_measures[0][0], score_CC2, interaction, gender, strain, "CC2", t, file]
                        
                        elif strain == "FCvsCC":
                            score_CC, score_FC = measures_FCvsCC(raw, network_measures[0][1], G, random_iterations ,exponent,tol,rescale)
                            df_split_temp_1.loc[len(df_split_temp_1)]=[network_measures[0][0], score_CC, interaction, gender, strain, "CC", t, file]
                            df_split_temp_2.loc[len(df_split_temp_2)]=[network_measures[0][0], score_FC, interaction, gender, strain, "FC", t, file]
                t+= 1
                if All:
                    new_list = (df_all_temp.iloc[0]).tolist()
                    if df_all_temp['Score'].mean() == 0:
                        print(df_all_temp['Score'])
                    new_list[1] = df_all_temp["Score"].mean()
                    print(new_list)

                    df_all_zscore.loc[len(df_all_zscore)]=new_list
                    df_all_temp = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File'])
                    df_all_zscore.to_csv("./data/temp_ALL_score.csv")

                else: 
                    new_list_1 = (df_split_temp_1.iloc[0].tolist())
                    new_list_1[1] = df_split_temp_1['Score'].mean()
                    df_split_zscore.loc[len(df_split_zscore)]=new_list_1
                    print(new_list_1)

                    new_list_2 = (df_split_temp_2.iloc[0].tolist())
                    new_list_2[1] = df_split_temp_2['Score'].mean()
                    df_split_zscore.loc[len(df_split_zscore)]=new_list_2
                    print(new_list_2)
                    print("-----------------------")
                                               
                    df_split_temp_1 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File'])
                                        
                    df_split_temp_2 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File'])
                    
                    df_split_zscore.to_csv("./data/temp_split_score.csv")
    

    if raw:
        type = "raw_"
    else:
        type = "_z-score_" + str(random_iterations) + "_"
    if network_measures[0][1] == "NA":
        type += "exponent_" + str(exponent)
    if All:
        df_all_zscore.to_csv("/raid/saraelshawa/October_Work/data/" + str(date) + "_" + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_ALL.csv")
        print("data saved in file: " + str("./data/" + str(date)  + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_ALL.csv"))
        return df_all_zscore
    
    df_split_zscore.to_csv("/raid/saraelshawa/October_Work/data/" + str(date) + "_"  + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_SPLIT.csv")
    print("data saved in file: " + str("./data/" + str(date) + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_SPLIT.csv"))
    return df_split_zscore


In [10]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['eigenvector centrality', nx.eigenvector_centrality_numpy]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  eigenvector centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality', 0.45229054027606036, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
['eigenvector centrality', 0.15188400690282158, 'Head-Tail', 'Male', 'NACvsCC', 'NAC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality', 0.3343136358968104, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NACv

['eigenvector centrality', 0.2059667827575724, 'Head-Tail', 'Female', 'CCvsCC', 'CC1', 16, 'fixed_Female-CCvsCC_3vs3Females_P60_CCvsCC_Group7Z_30minutes_interaction_cam_2_Wed_06_19_2019_mjpeg_tracks.mat']
['eigenvector centrality', 0.3909037987215471, 'Head-Tail', 'Female', 'CCvsCC', 'CC2', 16, 'fixed_Female-CCvsCC_3vs3Females_P60_CCvsCC_Group7Z_30minutes_interaction_cam_2_Wed_06_19_2019_mjpeg_tracks.mat']
-----------------------
fixed_Female-CCvsCC_3vs3Females_P60_CCvsCC_Group8M_interaction_30min_cam_2_Thu_08_01_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality', 0.4258305615464498, 'Head-Tail', 'Female', 'CCvsCC', 'CC1', 17, 'fixed_Female-CCvsCC_3vs3Females_P60_CCvsCC_Group8M_interaction_30min_cam_2_Thu_08_01_2019_mjpeg_tracks.mat']
['eigenvector centrality', 0.31044053475135996, 'Head-Tail', 'Female', 'CCvsCC', 'CC2', 17, 'fixed_Female-CCvsCC_3vs3Females_P60_CCvsCC_Group8M_interaction_30min_cam_2_Thu_08_01_2019_mjpeg_tracks.mat']
----------------------

['eigenvector centrality', 0.2998211242279906, 'Head-Tail', 'Male', 'CCvsCC', 'CC1', 31, 'fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group10X_30min_interaction__cam_2_Wed_06_12_2019_mjpeg_tracks.mat']
['eigenvector centrality', 0.2981633765676207, 'Head-Tail', 'Male', 'CCvsCC', 'CC2', 31, 'fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group10X_30min_interaction__cam_2_Wed_06_12_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group11Z_interaction_30min_cam_2_Wed_09_18_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality', 0.2586435990816285, 'Head-Tail', 'Male', 'CCvsCC', 'CC1', 32, 'fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group11Z_interaction_30min_cam_2_Wed_09_18_2019_mjpeg_tracks.mat']
['eigenvector centrality', 0.4473263320581037, 'Head-Tail', 'Male', 'CCvsCC', 'CC2', 32, 'fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group11Z_interaction_30min_cam_2_Wed_09_18_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-CCvsCC_3vs3Males_

['eigenvector centrality', 0.16104322894075532, 'Head-Tail', 'Male', 'FCvsCC', 'CC', 46, 'fixed_Male-FCvsCC_3vs3Males_P60_FCvsCC_Group4_0-30mins_interaction_cam_2_Wed_10_10_2018_mjpeg_tracks.mat']
['eigenvector centrality', 0.4399656516475772, 'Head-Tail', 'Male', 'FCvsCC', 'FC', 46, 'fixed_Male-FCvsCC_3vs3Males_P60_FCvsCC_Group4_0-30mins_interaction_cam_2_Wed_10_10_2018_mjpeg_tracks.mat']
-----------------------
fixed_Male-FCvsCC_3vs3Males_P60_FCvsCC_Group5_Interaction_30min_cam_2_Tue_01_08_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality', 0.3212540941027076, 'Head-Tail', 'Male', 'FCvsCC', 'CC', 47, 'fixed_Male-FCvsCC_3vs3Males_P60_FCvsCC_Group5_Interaction_30min_cam_2_Tue_01_08_2019_mjpeg_tracks.mat']
['eigenvector centrality', 0.3617247760305352, 'Head-Tail', 'Male', 'FCvsCC', 'FC', 47, 'fixed_Male-FCvsCC_3vs3Males_P60_FCvsCC_Group5_Interaction_30min_cam_2_Tue_01_08_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-FCvsCC_3vs3Males_P60_FCvsC

FileNotFoundError: [Errno 2] No such file or directory: '/raid/saraelshawa/October_Work/data/20191310_raw__Head-Tail_eigenvectorcentrality_SPLIT.csv'

In [10]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['eigenvector centrality', nx.eigenvector_centrality_numpy]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  False
Network Measure:  eigenvector centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality', 0.3075114348681118, 'Head-Tail', 'Male', 'NACvsCC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality', 0.3180151149093967, 'Head-Tail', 'Male', 'NACvsCC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['eigenvector centrality'

ArpackError: ARPACK error 1: The Schur form computed by LAPACK routine dlahqr could not be reordered by LAPACK routine dtrsen. Re-enter subroutine dneupd  with IPARAM(5)NCV and increase the size of the arrays DR and DI to have dimension at least dimension NCV and allocate at least NCV columns for Z. NOTE: Not necessary if Z and V share the same space. Please notify the authors if this erroroccurs.

In [31]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['indegree centrality', nx.in_degree_centrality]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  indegree centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['indegree centrality', 0.36296296296296304, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['indegree centrality', 0.2777777777777778, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./data/d

In [32]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['outdegree centrality', nx.out_degree_centrality]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  outdegree centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['outdegree centrality', 0.1888888888888889, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['outdegree centrality', 0.27619047619047615, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./dat

In [35]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['betweenness centrality', nx.betweenness_centrality]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  betweenness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['betweenness centrality', 0.061111111111111116, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['betweenness centrality', 0.10119047619047619, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.m

In [45]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['betweenness centrality', nx.betweenness_centrality]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  False
Network Measure:  betweenness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group1F_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group2G_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_M

In [37]:
%%time

import time
time.ctime()
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['closeness centrality', nx.closeness_centrality]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  closeness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['closeness centrality', 0.4405114638447972, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['closeness centrality', 0.34925170068027217, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./dat

In [46]:
%%time

import time
time.ctime()
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['closeness centrality', nx.closeness_centrality]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  False
Network Measure:  closeness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group1F_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group2G_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Mal

In [44]:
%%time

import time
time.ctime()
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for t in treatments:
    x = [0, 1, -1, 0.1, -0.1]
    for exponent in x:
        df_zscore_ev_split = all_treatment_analysis(t, False, [['power centrality', "NA"]], False, exponent, 1e-07,True, True, 0)

Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  power centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['power centrality', 0.11805555555555555, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['power centrality', 0.17261904761904762, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
--------------------

In [39]:
%%time

# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['clustering', nx.clustering]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  clustering
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['clustering', 0.22366255144032923, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
['clustering', 0.17125898792565458, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 0, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat']
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat


In [47]:
%%time

# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['clustering', nx.clustering]], False, 0, 1e-07,True, True, 0)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  False
Network Measure:  clustering
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group1F_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group2G_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
-----------------------
fixed_Male-NacvsCC_

# Z-scores, 2500 random iterations for betweenness centrality, closeness centrality

In [55]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['betweenness centrality', nx.betweenness_centrality]], False, 0, 1e-07,True, False, 2500)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  False
Split groups calculation:  True
Network Measure:  betweenness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['betweenness centrality', -0.5855877962505479, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
['betweenness centrality', -0.2813655990317693, 'Head-Tail', 'Male', 'NACvsCC', 'NAC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['betweenness centrality', 0.3842326141409715, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NAC

In [56]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['closeness centrality', nx.closeness_centrality]], False, 0, 1e-07,True, False, 2500)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  False
Split groups calculation:  True
Network Measure:  closeness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['closeness centrality', 1.148034824372085, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
['closeness centrality', -1.578408425489076, 'Head-Tail', 'Male', 'NACvsCC', 'NAC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['closeness centrality', 0.03808203887769899, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group

In [57]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['clustering', nx.clustering]], False, 0, 1e-07,True, False, 2500)

['./data/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  False
Split groups calculation:  True
Network Measure:  clustering
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['clustering', 0.08118766546292372, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
['clustering', -0.033114529497129425, 'Head-Tail', 'Male', 'NACvsCC', 'NAC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Tail_20191013.mat
['clustering', -0.42164629160671346, 'Head-Tail', 'Male', 'NACvsCC', 'CC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_

In [7]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Head_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['clustering', nx.clustering]], False, 0, 1e-07,True, False, 2500)

['./data/dbs_Head-Head_20191013.mat']
Properties of current analysis:
Raw:  False
Split groups calculation:  True
Network Measure:  clustering
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Head_20191013.mat
['clustering', -0.5422465648153867, 'Head-Head', 'Male', 'NACvsCC', 'CC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
['clustering', -0.6973173065687784, 'Head-Head', 'Male', 'NACvsCC', 'NAC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group4N_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Head_20191013.mat
['clustering', -0.22799452130103734, 'Head-Head', 'Male', 'NACvsCC', 'CC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group4N_interaction_30min_cam_2_Tue_08_06

In [8]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Head_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['betweenness centrality', nx.betweenness_centrality]], False, 0, 1e-07,True, False, 2500)

['./data/dbs_Head-Head_20191013.mat']
Properties of current analysis:
Raw:  False
Split groups calculation:  True
Network Measure:  betweenness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Head_20191013.mat
['betweenness centrality', -0.6021668131317253, 'Head-Head', 'Male', 'NACvsCC', 'CC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
['betweenness centrality', 0.46058518925666403, 'Head-Head', 'Male', 'NACvsCC', 'NAC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group4N_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Head_20191013.mat
['betweenness centrality', -0.1418343828731094, 'Head-Head', 'Male', 'NACvsCC', 'CC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NA

In [3]:
!pwd

/raid/saraelshawa/October_Work


In [9]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Head_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['closeness centrality', nx.closeness_centrality]], False, 0, 1e-07,True, False, 2500)

['./data/dbs_Head-Head_20191013.mat']
Properties of current analysis:
Raw:  False
Split groups calculation:  True
Network Measure:  closeness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
./data/dbs_Head-Head_20191013.mat
['closeness centrality', -0.4541034315231831, 'Head-Head', 'Male', 'NACvsCC', 'CC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
['closeness centrality', 0.046131787803536245, 'Head-Head', 'Male', 'NACvsCC', 'NAC', 1, 'fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat']
-----------------------
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group4N_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
./data/dbs_Head-Head_20191013.mat
['closeness centrality', -0.9782846086589001, 'Head-Head', 'Male', 'NACvsCC', 'CC', 2, 'fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_G

In [2]:
# %%time
# # def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# # treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
# treatments = [["./data/dbs_Head-Head_20191013.mat"]]
# for x in treatments:
#     print(x)
#     df_zscore_ev_split = all_treatment_analysis(x, False, [['eigenvector centrality', nx.eigenvector_centrality]], False, 0, 1e-07,True, False, 2500)

In [1]:
# %%time
# # def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# # treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
# treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
# for x in treatments:
#     print(x)
#     df_zscore_ev_split = all_treatment_analysis(x, False, [['eigenvector centrality', nx.eigenvector_centrality]], False, 0, 1e-07,True, False, 2500)

# power centrality z-score

In [42]:
din = [0,1,2,3,4,5]
dout = [5,2,4,3,1,0]
results=np.zeros((6,1000))
for i in range(100):
    done=False
    while not done:
        try:
            #results[:,i] gets column i
            results[:,i]=np.array(list(bonacich_power_centrality(nx.directed_configuration_model(din,dout, create_using=nx.DiGraph()), 1, 1e-07, False).values()))
            done=True
        except:
            False

print(np.mean((results[:,1]-np.mean(results[:,1:],1))/np.std(results[:,1:])))

[ 0.1699974  -4.37496059  2.61839886 -5.71057608  0.92970654  0.        ]


In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, 0, 1e-07,True, False, 2500)

In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, -1, 1e-07,True, False, 2500)

In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, 1, 1e-07,True, False, 2500)

In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, 0.1, 1e-07,True, False, 2500)